In [6]:
import numpy as np
from itertools import combinations

class Apriori:
    def __init__(self, min_support=0.5, min_confidence=0.7):
        """
        Initialize Apriori algorithm parameters.

        Parameters:
        - min_support: Minimum support threshold (float between 0 and 1)
        - min_confidence: Minimum confidence threshold (float between 0 and 1)
        """
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.frequent_itemsets = None
        self.association_rules = None

    def _get_support(self, itemset, transactions):
        """Calculate support for an itemset."""
        count = 0
        for transaction in transactions:
            if all(item in transaction for item in itemset):
                count += 1
        return count / len(transactions)

    def _generate_candidates(self, itemsets, length):
        """Generate candidate itemsets of specified length."""
        items = sorted(set(item for itemset in itemsets for item in itemset))
        return list(combinations(items, length))

    def _prune_candidates(self, candidates, prev_itemsets, length):
        """Prune candidates using the Apriori property."""
        pruned = []
        for candidate in candidates:
            subsets = combinations(candidate, length-1)
            if all(subset in prev_itemsets for subset in subsets):
                pruned.append(candidate)
        return pruned

    def fit(self, transactions):
        """
        Mine frequent itemsets and generate association rules.

        Parameters:
        - transactions: List of transactions (each transaction is a list of items)
        """
        # Convert transactions to frozensets for faster operations
        transactions = [frozenset(t) for t in transactions]

        # Find frequent itemsets
        self.frequent_itemsets = {}
        length = 1

        # Get frequent items of length 1
        items = set(item for transaction in transactions for item in transaction)
        candidates = [frozenset([item]) for item in items]

        while candidates:
            # Calculate support for candidates
            frequent_candidates = []
            for candidate in candidates:
                support = self._get_support(candidate, transactions)
                if support >= self.min_support:
                    frequent_candidates.append((candidate, support))

            # Store frequent itemsets of current length
            if frequent_candidates:
                self.frequent_itemsets[length] = frequent_candidates
                length += 1

                # Generate next level candidates
                candidates = self._generate_candidates(
                    [itemset for itemset, support in frequent_candidates],
                    length
                )

                # Prune candidates
                prev_itemsets = set(itemset for itemset, support in frequent_candidates)
                candidates = self._prune_candidates(candidates, prev_itemsets, length-1)
            else:
                candidates = None

        # Generate association rules
        self.association_rules = []
        for length, itemsets in self.frequent_itemsets.items():
            if length == 1:
                continue

            for itemset, support in itemsets:
                # Generate all possible non-empty subsets
                for i in range(1, length):
                    for antecedent in combinations(itemset, i):
                        antecedent = frozenset(antecedent)
                        consequent = itemset - antecedent

                        # Calculate confidence
                        antecedent_support = self._get_support(antecedent, transactions)
                        confidence = support / antecedent_support

                        if confidence >= self.min_confidence:
                            self.association_rules.append({
                                'antecedent': antecedent,
                                'consequent': consequent,
                                'support': support,
                                'confidence': confidence
                            })

    def get_frequent_itemsets(self, length=None):
        """
        Get frequent itemsets.

        Parameters:
        - length: Specific length of itemsets to return (None for all)

        Returns:
        - List of frequent itemsets with their support
        """
        if length is None:
            return [itemset for l in self.frequent_itemsets.values() for itemset in l]
        return self.frequent_itemsets.get(length, [])

    def get_association_rules(self):
        """Get association rules meeting min confidence threshold."""
        return self.association_rules

# Example usage
if __name__ == "__main__":
    # Sample transaction data (market basket data)
    transactions = [
        ['bread', 'milk'],
        ['bread', 'diapers', 'beer', 'eggs'],
        ['milk', 'diapers', 'beer', 'cola'],
        ['bread', 'milk', 'diapers', 'beer'],
        ['bread', 'milk', 'diapers', 'cola']
    ]

    # Initialize and run Apriori algorithm
    apriori = Apriori(min_support=0.4, min_confidence=0.6)
    apriori.fit(transactions)

    # Display results
    print("Frequent Itemsets:")
    for length, itemsets in apriori.frequent_itemsets.items():
        print(f"\nLength {length}:")
        for itemset, support in itemsets:
            print(f"{set(itemset)}: support = {support:.2f}")

    print("\nAssociation Rules:")
    for rule in apriori.get_association_rules():
        print(f"{set(rule['antecedent'])} => {set(rule['consequent'])} "
              f"(support={rule['support']:.2f}, confidence={rule['confidence']:.2f})")

Frequent Itemsets:

Length 1:
{'beer'}: support = 0.60
{'diapers'}: support = 0.80
{'milk'}: support = 0.80
{'cola'}: support = 0.40
{'bread'}: support = 0.80

Association Rules:
